In [4]:
# 2_train_model.py
from clearml import Task, Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib # для сохранения модели

# --- Шаг 1: Инициализируем задачу в ClearML ---
task = Task.init(
    project_name='examples-mnist',
    task_name='train_logistic_regression',
    output_uri=True # Очень важно! Говорит ClearML сохранять артефакты (модели)
)

# --- Шаг 2: Получаем данные из ClearML Data ---
print('Получаем последнюю версию датасета...')
# Это скачает данные, если их нет локально
dataset = Dataset.get(dataset_project='examples-mnist', dataset_name='mnist_dataset')
local_dataset_path = dataset.get_local_copy()

print(f'Данные находятся в: {local_dataset_path}')
X = np.load(f'{local_dataset_path}/mnist_features.npy', allow_pickle=True)
y = np.load(f'{local_dataset_path}/mnist_labels.npy', allow_pickle=True)
print(f'dataset X: {X}')
print(f'dataset Y: {y}')
# --- Шаг 3: Определяем параметры ---
# ClearML автоматически их подхватит
params = {
    'test_size': 0.2,
    'random_state': 42,
    'C': 0.1
}
# Явно подключаем параметры к задаче для лучшей организации
task.connect(params)

# --- Шаг 4: Обучение модели ---
print('Разделяем данные и обучаем модель...')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=params['test_size'], random_state=params['random_state']
)

model = LogisticRegression(C=params['C'], max_iter=1000) # Простая модель
model.fit(X_train, y_train)

# --- Шаг 5: Оценка и логирование метрик ---
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy = {acc}')

# Логируем метрику в ClearML, чтобы видеть красивые графики
task.get_logger().report_scalar(
    title='performance', series='accuracy', value=acc, iteration=1
)

# --- Шаг 6: Сохраняем модель ---
# ClearML автоматически найдет этот файл и загрузит его
joblib.dump(model, 'model.pkl')

print('Задача выполнена! Модель сохранена и загружена в ClearML.')

Получаем последнюю версию датасета...
2025-07-23 12:59:24,956 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
Данные находятся в: /home/zazaka/.clearml/cache/storage_manager/datasets/ds_bf0142140fc746f5b72781cc327af8b9
dataset X: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Разделяем данные и обучаем модель...


/home/zazaka/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy = 0.9161428571428571
Задача выполнена! Модель сохранена и загружена в ClearML.
